In [1]:
from pathlib import Path
import shutil
import numpy as np

In [2]:
RUN_DIR =    Path("/project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/5ns/runs")
MOLD_DIR =   Path("/project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/5ns/mold")
INIT_DIR =   Path("/project2/andrewferguson/armin/FE_DATA/GHGGF_FEN/complex/usample_2d/prep/distances/runs")
NUM_SUB = 4
NWINDOWS_DIST = 12
NWINDOWS_ANG  = 18
INIT_FLAG = False

# Preparations

In [3]:
def write_plumed(id, dist, ang, k_d, k_a, plumed_f):
    plumed_cnt = [ "WHOLEMOLECULES ENTITY0=99-151 ENTITY1=1-98\n",
                   "# DEFINITIONS\n",
                   "MOLcom: COM ATOMS=99-151\n",
                   "PCCcom: COM ATOMS=1-98\n",
                   "# COLVARS\n",
                   "dcom: DISTANCE ATOMS=MOLcom,PCCcom\n",
                   "ang: ANGLE ATOMS=5,8,101,129\n",
                   "v1: DISTANCE ATOMS=5,8 SCALED_COMPONENTS NOPBC\n",
                   "v2: DISTANCE ATOMS=101,129 SCALED_COMPONENTS NOPBC\n",
                   "vr: DISTANCE ATOMS=PCCcom,2 SCALED_COMPONENTS NOPBC\n",
                   "crossx: CUSTOM ...\n",
                   "\tARG=v1.b,v1.c,v2.b,v2.c\n",
                   "\tVAR=y1,z1,y2,z2\n",
                   "\tFUNC=y1*z2-y2*z1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crossy: CUSTOM ...\n",
                   "\tARG=v1.a,v1.c,v2.a,v2.c\n",
                   "\tVAR=x1,z1,x2,z2\n",
                   "\tFUNC=x2*z1-x1*z2\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crossz: CUSTOM ...\n",
                   "\tARG=v1.a,v1.b,v2.a,v2.b\n",
                   "\tVAR=x1,y1,x2,y2\n",
                   "\tFUNC=x1*y2-x2*y1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "crosss: CUSTOM ...\n",
                   "\tARG=crossx,crossy,crossz\n",
                   "\tVAR=x,y,z\n",
                   "\tFUNC=sqrt(x*x+y*y+z*z)\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refx: CUSTOM ...\n",
                   "\tARG=v1.b,v1.c,vr.b,vr.c\n",
                   "\tVAR=y1,z1,y2,z2\n",
                   "\tFUNC=y1*z2-y2*z1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refy: CUSTOM ...\n",
                   "\tARG=v1.a,v1.c,vr.a,vr.c\n",
                   "\tVAR=x1,z1,x2,z2\n",
                   "\tFUNC=x2*z1-x1*z2\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refz: CUSTOM ...\n",
                   "\tARG=v1.a,v1.b,vr.a,vr.b\n",
                   "\tVAR=x1,y1,x2,y2\n",
                   "\tFUNC=x1*y2-x2*y1\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "refs: CUSTOM ...\n",
                   "\tARG=refx,refy,refz\n",
                   "\tVAR=x,y,z\n",
                   "\tFUNC=sqrt(x*x+y*y+z*z)\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "dc: CUSTOM ...\n",
                   "\tARG=crossx,crossy,crossz,refx,refy,refz,crosss,refs\n",
                   "\tVAR=x1,y1,z1,x2,y2,z2,A,B\n",
                   "\tFUNC=(x1*x2+y1*y2+z1*z2)/A/B\n",
                   "\tPERIODIC=NO\n",
                   "...\n",
                   "# BIAS\n",
                   f"udb: RESTRAINT ARG=dcom KAPPA={k_d} AT={dist}\n",
                   f"uab: RESTRAINT ARG=ang KAPPA={k_a} AT={ang}\n",
                   "\n",
                   f"PRINT ARG=dcom,ang,dc,udb.bias,uab.bias STRIDE=100 FILE=us_{id}.dat\n",
                    ]
    with open(plumed_f, 'w') as f:
        f.writelines(plumed_cnt)

def write_sub(f_name, init_dir, ang_id, dist_id):
    with open(f_name, 'r' ) as f:
        f_cnt = f.readlines()
    line = f_cnt[-3]
    line = line.replace("$ANG_DIR", str(init_dir/f"{ang_id}"/f"{dist_id}"))
    f_cnt[-3] = line
    with open(f_name, 'w' ) as f:
        f.writelines(f_cnt)
    return None

In [4]:
DISTS = np.linspace(0, 2.5, NWINDOWS_DIST)
ANGS1 = np.linspace(0, np.pi, NWINDOWS_ANG)
ANGS2 = ANGS1[1:-1]
ANGS2 = ANGS2[::-1]
ANGS = np.concatenate((ANGS1, ANGS2), axis=0)

In [5]:
if INIT_FLAG:
    for i, ang in enumerate(ANGS):
        for j, dist in enumerate(DISTS):
            sim_dir = RUN_DIR/f"{i}_{j}"
            if sim_dir.exists():
                is_empty = not any(sim_dir.iterdir())
                if not is_empty:
                    raise RuntimeError(f"Directory {i}/{j} already exists and is not empty.")
            else:
                sim_dir.mkdir()
    
            for file in MOLD_DIR.iterdir():
                shutil.copy(file, sim_dir/file.parts[-1])
            
            plumed_f = sim_dir/"plumed_us.dat"
            write_plumed(f"{i}_{j}", dist, ang, 250.00, 250.00, plumed_f)
    
            sub_f = sim_dir/f"sub_mdrun_plumed.sh"
            write_sub(sub_f, INIT_DIR, i, j)

# Submission

Just a little context manager for changing directories

In [10]:
import os
import contextlib

@contextlib.contextmanager
def working_directory(path):
    """Changes working directory and returns to previous on exit."""
    prev_cwd = Path.cwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(prev_cwd)

In [11]:
import subprocess
import multiprocessing

def call_func(ids):
   i, j = ids
   with working_directory(RUN_DIR/f"{i}_{j}"):
      gro_dir = Path.cwd()/"md.gro"
      if gro_dir.exists():
         print(f"{i}_{j}: Skipping")
      else:
         print(f"{i}_{j}: ", end="")
         subprocess.call("sbatch -J GHGGF_us --wait sub_mdrun_plumed.sh", shell=True)
   return None

ids = []
for i in range(len(ANGS)):
   for j in range(len(DISTS)):
      ids.append((i, j))


with multiprocessing.Pool(NUM_SUB) as pool:
   pool.map(call_func, ids)

4_4: Skipping
0_0: Skipping
0_1: Skipping
6_6: Skipping2_2: Skipping

6_7: Skipping
6_8: 4_5: Skipping
0_2: Skipping
2_3: Skipping
0_3: Skipping
0_4: Skipping
0_5: Skipping
0_6: Skipping
2_4: Skipping
0_7: Skipping
2_5: Skipping
4_6: Skipping
2_6: Skipping
2_7: Skipping
2_8: Skipping0_8: Skipping

2_9: Skipping0_9: Skipping

2_10: Skipping0_10: Skipping

2_11: Skipping
0_11: Skipping
3_0: Skipping1_0: Skipping

3_1: Skipping
4_7: Skipping3_2: Skipping

3_3: 4_8: Skipping
4_9: Skipping
4_10: Skipping
1_1: Skipping4_11: Skipping

5_0: Skipping
5_1: Skipping
5_2: Skipping
5_3: Skipping
5_4: Skipping1_2: Skipping

1_3: Skipping
1_4: Skipping
1_5: Skipping
5_5: Skipping1_6: Skipping

5_6: Skipping
5_7: Skipping
5_8: Skipping
5_9: Skipping
5_10: Skipping
5_11: 1_7: Skipping
1_8: Skipping
1_9: Submitted batch job 31102981
Submitted batch job 31102982
Submitted batch job 31102983
Submitted batch job 31102984
3_4: Skipping
3_5: Skipping
3_6: Skipping
3_7: Skipping
3_8: Skipping
3_9: Skipping
3_

In [12]:
import os
import asyncio
import telegram

async def notif(message):
    bot = telegram.Bot("5848656908:AAHqbFxp3yrPyZNOSgeUUP59aD8_5pB3lLc")
    async with bot:
        await bot.send_message(chat_id=149407364, text=message)


await notif(f"{os.getcwd()} is done.")